# Optymalizacja przejazdów komunikacją miejską
Jakub Czajkowski 272709

# Opis teoretyczny

## Algorytm Dijkstry
Podstawowy algorytm do znajdowania najkrótszych ścieżek w grafie ważonym o nieujemnych wagach. Wykorzystuje zachłanną strategię, gdzie iteracyjnie wybiera wierzchołek o najmniejszej dotychczasowej odległości i aktualizuje odległości do jego sąsiadów. Złożoność: O((V + E) log V). 

## Algorytm A*
Rozszerzenie Dijkstry wykorzystujące funkcję heurystyczną h(n) do szacowania odległości do celu. Funkcja oceny f(n) = g(n) + h(n), gdzie g(n) to dotychczasowy koszt, a h(n) to szacowany koszt do celu. Kluczowym wymogiem jest, aby h(n) nie przeszacowywała rzeczywistego kosztu (tzw. dopuszczalna heurystyka).

## Tabu Search
Metaheurystyka do rozwiązywania problemów optymalizacyjnych, szczególnie przydatna dla problemu komiwojażera. Wykorzystuje pamięć krótkoterminową (listę tabu) do unikania zapętleń i mechanizmy takie jak aspiracja do umożliwienia "zakazanych" ruchów, jeśli prowadzą do lepszych rozwiązań.


# Implementacja
## Struktura projektu
Projekt składa się z następujących modułów:

- `main.py`: główny skrypt uruchomieniowy
- `utils.py`: funkcje do wczytywania danych
- `algorithms.py`: implementacje algorytmów Dijkstry i A*
- `tsp.py`: implementacja algorytmu Tabu Search dla problemu wielu przystanków
- `graph.py`: reprezentacja grafu
- `constants.py`: stałe używane w projekcie

## Najważniejsze fragmenty kodu
### Wczytywanie danych
W celu szybszego przetwarzania danyc wczytujemy je z pliku CSV. Wykorzystujemy bibliotekę `pandas` do wczytania danych i przetworzenia ich na odpowiedni format. Pomijamy duplikaty i konwertujemy nazwy przystanków na małe litery, aby uniknąć problemów z porównywaniem. W celu przyspieszenia działania programu, graf jest zapisywany w pliku binarnym przy użyciu `pickle`. W przypadku braku pliku, graf jest tworzony na nowo.
```python
def load_data():
    data = pd.read_csv(data_file_path, dtype={2: str}, usecols=csv_columns).drop_duplicates()
    data["start_stop"] = data["start_stop"].str.lower()
    data["end_stop"] = data["end_stop"].str.lower()
    return data


def load_graph():
    if os.path.exists(graph_file):
        try:
            with open(graph_file, 'rb') as f:
                graph = pickle.load(f)
            print("Graph loaded from file.")
        except (pickle.UnpicklingError, EOFError, FileNotFoundError):
            print("Failed to load graph from file, creating a new one.")
            graph = create_and_pickle_graph()
    else:
        print("Graph file not found, creating a new one.")
        graph = create_and_pickle_graph()

    return graph
```

### Graf
Graf jest reprezentowany jako słownik, gdzie kluczem jest wierzchołek (przystanek), a wartością lista krawędzi (sąsiadów) z wagami. Krawędzie są reprezentowane jako krotki `(sąsiad, waga)`. Wykorzystujemy algorytm Dijkstry do obliczenia najkrótszych ścieżek między przystankami.

Do najważniejszych funkcji w klasie `Graph` należą funkje `get_time_cost` i `get_line_cost`, które obliczają koszty przejazdu między przystankami. Funkcja `get_distance` oblicza odległość między dwoma przystankami na podstawie ich współrzędnych geograficznych (wykorzystywane w algorytmie A*).

```python
class Graph:
    def __init__(self):
        self.nodes = dict()
        self.edges = dict()

    def get_time_cost(self, start, end, time: datetime.datetime, last_line, used_lines):
        possible_paths = self.nodes[start].get_sorted_edges(end)
        if not possible_paths:
            return float('inf'), None

        soonest_id = self.find_first_id_after(possible_paths, time)
        if soonest_id is None:
            soonest_id = 0
        cost = possible_paths[soonest_id].get_travel_time() + abs(
            (possible_paths[soonest_id].arrival_time - time).seconds / 60.0)
        if possible_paths[soonest_id].line != last_line:
            cost += 0
        return cost, possible_paths[soonest_id]

    def get_line_cost(self, start, end, time, last_line, used_lines):
        possible_paths = self.nodes[start].get_sorted_edges(end)
        if not possible_paths:
            return float('inf'), None

        soonest_id = self.find_first_id_after(possible_paths, time)
        if soonest_id is None:
            soonest_id = 0

        travel_time = possible_paths[soonest_id].get_travel_time()
        waiting_time = abs((possible_paths[soonest_id].departure_time - time).seconds / 60.0)

        cost = travel_time + waiting_time
        if possible_paths[soonest_id].line != last_line and last_line is not None:
            cost += 100 * used_lines

        return cost, possible_paths[soonest_id]

    def get_distance(self, from_stop, to_stop):
        if from_stop not in self.nodes or to_stop not in self.nodes:
            return 0.0
        lat1, lon1 = self.nodes[from_stop].get_avg_location()
        lat2, lon2 = self.nodes[to_stop].get_avg_location()
        return (abs(lon2 - lon1) + abs(lat2 - lat1)) * km_per_deg * mins_per_km
```
### Dijkstra
*Ze względu na dużą ilość kodu, postanowiono nie zamieszczać kodu do algorytmów Dijkstry, A\* oraz Tabu Search w tym miejscu. Kod można znaleźć w pliku `algorithms.py`.*


Najważniejsze cechy:

- Wykorzystanie kolejki priorytetowej do wyboru najbliższego wierzchołka
- Funkcja kosztu jako parametr (umożliwia optymalizację czasu lub liczby przesiadek)
- Uwzględnienie czasu oczekiwania na przystanku
- Rekonstrukcja ścieżki z poprzedników

### A*
Najważniejsze cechy:

- Wykorzystanie funkcji heurystycznej bazującej na odległości geograficznej
- Skalowanie heurystyki (0.7) dla zapewnienia dopuszczalności
- Optymalizacja z wykorzystaniem zbioru odwiedzonych wierzchołków
- Szacowanie odległości za pomocą Manhattan distance (dla przystanków)

### Tabu Search
Najważniejsze funkcje:

- calculate_initial_solution() - tworzenie początkowego rozwiązania (greedy lub wszystkie permutacje dla małych problemów)
-  generate_neighbors() - generowanie sąsiednich rozwiązań przez zamianę przystanków
- calculate_route_cost() - obliczanie kosztu trasy z wczesnym zakończeniem
- get_tabu_list_size() - dynamiczne dostosowanie rozmiaru listy tabu

Kluczowe usprawnienia:

Dynamiczna lista tabu: rozmiar dostosowany do liczby przystanków
Kryterium aspiracji: umożliwienie "zakazanych" ruchów, jeśli poprawiają najlepsze rozwiązanie
Zaawansowane próbkowanie sąsiedztwa: efektywny wybór sąsiadów do oceny
Buforowanie tras: przechowywanie obliczonych wcześniej segmentów tras
Wczesne zakończenie obliczania kosztu: przerwanie jeśli aktualna trasa jest już gorsza od najlepszej
Adaptacja parametrów do rozmiaru problemu: mniej iteracji dla małych problemów

# Optymalizacja kodu

Po wprowadzeniu usprawnień, algorytmy wykorzystują wyszukiwanie binarne do znajdowania najszybciej odjeżdżającego pojazdu. Implementacja opiera się na gotowej funkcji bisect z biblioteki bisect. Dodatkowo, graf jest tworzony tylko raz i przechowywany w pamięci, co znacząco przyspiesza odczyt danych. Wprowadzono również cachowanie tras w Tabu Search, co pozwala na uniknięcie ponownego obliczania kosztów dla tych samych tras. Wykorzystano również dynamiczne dostosowanie rozmiaru listy tabu do liczby przystanków, co pozwala na lepsze dopasowanie do problemu.

# Wyniki

Algorytmy zostały przetestowane dla trasy Pl. Grunwaldzki - Wrocławski Park Przemysłowy. Wyjątek stanowi algorytm Tabu search, który obejmował dodatkowo przystanki: Litewska, Poprzeczna oraz Galeria Dominikańska. Przypadki testowe zostały przetestowane w pliku `main.py`.

|                   | Dijkstra - czas | Dijkstra - przesiadki | A* - czas | A* przesiadki | Tabu Search |
|:-----------------:|:---------------:|:---------------------:|:---------:|:-------------:|:-----------:|
|   Czas wykonania  |      0.4 s      |        0.2815 s       |  0.1123 s |    0.1628 s   |  5.8188 s   |
|   Czas przejazdu  |      36 min     |         47 min        |   36 min  |     47 min    |  2h 13 min  |
| Liczba przesiadek |        7        |           6           |     7     |       8       |     28      |

Na podstawie przedstawionych danych widać, że algorytmy Dijkstra i A* optymalizujące czas przejazdu dają identyczne wyniki (36 minut), przy czym A* wykonuje się znacznie szybciej (0.1123 s wobec 0.4 s). Warianty tych algorytmów optymalizujące przesiadki generują dłuższe czasy przejazdu, co wskazuje na kompromis między czasem a komfortem podróży. Dijkstra minimalizująca przesiadki radzi sobie lepiej (6 przesiadek) niż A* (8 przesiadek). 

Algorytm Tabu Search wypada najgorzej zarówno pod względem czasu wykonania (5.8188 s), czasu przejazdu (1h 53 min), jak i liczby przesiadek (23).  Optymalizuje on jednak trasę pomiędzy 5, a nie 2 przystankami.

# Wykorzystane biblioteki zewnętrzne

- `datetime` - obsługa i manipulacja czasem
- `heapq` - implementacja kolejki priorytetowej
- `time` - pomiar czasu wykonania
- `sys` - operacje wejścia/wyjścia
- `random` - generowanie losowych rozwiązań
- `math` - obliczenia matematyczne

# Napotkane problemy implementacyjne

- Źle sformatowane godziny nocne - można było zauważyć godzinę 27 lub nawet 30, należało więc najpierw dokonać operacji modulo dla wszystkich danych.
- Obsługiwanie przejazdów zaczynających się przed północą o kończących po. Pojawił się problem z określeniem czy to, że jedna godzina jest mniejsza od drugiej wynika z tego, że następuje wcześniej, czy z tego, że następuje kolejnego dnia po północy. Na szczęście biblioteka datetime okazała się pomocna oferując ujemne różnice czasowe, np -2 minuty dla operacji 0:01 - 23:59.
- Zdefiniowanie zadowalającej funkcji kosztu oraz heurystyki, która pozwoliła na wybranie najlepszego rozwiązania. Metoda prób i błędów przy określaniu parametrów była niezwykle czasochłonna. Mimo wszystko nie udało ustawić ich na optymalnym poziomie.


# Podsumowanie

Zaimplementowano i przetestowano algorytmy wyszukiwania optymalnych tras w sieci komunikacji publicznej. Wyniki pokazują, że A* jest znacząco szybszy od Dijkstry, a modyfikacje Tabu Search pozwoliły na poprawę jakości rozwiązań i skrócenie czasu obliczeń.
Dalsze kierunki rozwoju mogłyby obejmować implementację innych metaheurystyk lub optymalizację wielokryterialną.